# The English Premier League (EPL) Standings 

I created a function to develope an analytical understanding of the EPL's game results by creating a comprehensive function in R that retrieves and processes data about EPL match results from the web.

The function called EPL_Standings that takes two parameters - date and season - and returns the league standings for the specific date and season that a user specifies. The function is capable of handling data variations and inconsistencies.

In [ ]:
# calling important libraries

library(readr)
library(tidyverse)
library(dplyr)
library(stringr)


In [3]:
EPL_Standings <- function(date,season){
  # first using (if else) statement to read the right season
  if(season == '2023/24'){
    # read in the data using readr library
    df <- read_csv(url('https://www.football-data.co.uk/mmz4281/2324/E0.csv'))
  } else if (season == '2022/23'){
    df <- read_csv(url('https://www.football-data.co.uk/mmz4281/2223/E0.csv'))
  } else if (season == '2021/22'){
    df <- read_csv(url('https://www.football-data.co.uk/mmz4281/2122/E0.csv'))
  } else {
    message('Invalid season')
  }
  
  epl <- df %>%
      # select the necessary columns 
    select(Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR) %>%
      # change the data type of the date column into a date format using as.Date() 
      # and then change it into american date format by using format() function
    mutate(Date = as.Date(Date, format = "%d/%m/%Y"),
           Date = format(Date, "%m/%d/%Y")) %>%
      # After that, the data will be filtered to include everything up to the date 
      # mentioned in the function's input
    filter(Date <= date)
  
      # calculating the home record for each team: Wins-Losses-Ties , Home Goals(HGS)
      # and Home Goals Allowed (GAH)
    Home_epl <- epl %>%
    filter(Date <= date) %>%
    group_by(Team = HomeTeam) %>%
    summarise(
      HomeRec = paste( 
        sum(FTR == "H"),
        sum(FTR == "A"),
        sum(FTR == "D"), sep = '-'),
      HGS = sum(FTHG),
      GAH = sum(FTAG)
    ) 
      
    # calculating the away record for each team: Wins-Losses-Ties , Away Goals(AGS)
    # and Away Goals Allowed (GAA) 
    
    Away_epl <- epl %>%
    filter(Date <= date) %>%
    group_by(AwayTeam) %>%
    summarise(
      AwayRec = paste( 
        sum(FTR == "A"),
        sum(FTR == "H"),
        sum(FTR == "D"), sep = '-'),
      AGS = sum(FTAG),
      GAA = sum(FTHG)
    ) 
    
    
    # Now combine the Away and Home calculations using cbind() fuction
        # I excluded Away team to avoid duplicate column
    
    new_epl <- cbind(Home_epl,select(Away_epl,-AwayTeam))
    
    
    # now applying the other calculations on the Home and Away results for 
    # each team
    record <- new_epl %>%
      # first separate the home and away wins and losses and draws into a columns
    separate(HomeRec, into = c("HW", "HL", "HD"), sep = "-") %>%
    separate(AwayRec, into = c("AW", "AL", "AD"), sep = "-") %>%
      # then using across() which applies the as.numeric function 
      # to all columns that start with "H" and "A" to change the above separated
      # columns into a numeric data type to do the necessary calculations
    mutate(across(starts_with("H"), as.numeric),
           across(starts_with("A"), as.numeric))%>%
      
      # now I calculated for each team the : 
         # Record that has total (wins-losses-ties)
         # Matches Played
         # Points which is 3 multiplied by the total wins and one multiplied by total draws
         # Points per Match (PPM)
         # Point percentage (PtPct)
         # goals scored (GS)
         # goals scored per match (GSM)
         # goals allowed (GA)
         # goals allowed per match (GAM) 
    mutate(Record = paste(HW + AW,
                          HL + AL,
                          HD + AD, sep = '-'),
           MatchesPlayed = HW + AW +
             HL + AL + HD + AD,
           Points = 3*(HW + AW)+ 1*(HD + AD),
           PPM = round(Points / MatchesPlayed , digits = 2),
           PtPct = round (Points / 3* MatchesPlayed , digits = 2),
           GS = HGS + AGS,
           GSM = round(GS /MatchesPlayed , digits = 2),
           GA = GAH + GAA,
           GAM = round(GA /MatchesPlayed , digits = 2)
    ) 
    
    # then concatenating the two data frames : 
       # 1- new_epl that has the combined home and away results for each team
       # 2- record that has all of the above calculations
    # I also used cbind() function and selected the columns as needed
    new_epl <- cbind(
      new_epl %>% select(Team, HomeRec, AwayRec) ,
      record  %>% select(Record, MatchesPlayed, Points, PPM, PtPct, 
                         GS, GSM, GA, GAM)) %>%
      # then arranging the results in a descending order by :
          # Points per match
          # I extracted the total Wins from the Record column using str_extract()
          # Goals scored per match
      # and ascending order by goals allowed per match
      arrange(desc(PPM),desc(as.numeric(str_extract(Record, "\\d+"))),
              desc(GSM), GAM)
    
    return(new_epl)
}

In [4]:
result <- EPL_Standings('12/12/2023','2023/24')

Rows: 169 Columns: 106
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (7): Div, Date, HomeTeam, AwayTeam, FTR, HTR, Referee
dbl  (98): FTHG, FTAG, HTHG, HTAG, HS, AS, HST, AST, HF, AF, HC, AC, HY, AY,...
time  (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
head(result)

,Team,HomeRec,AwayRec,Record,MatchesPlayed,Points,PPM,PtPct,GS,GSM,GA,GAM
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Liverpool,7-0-0,4-1-4,11-1-4,16,37,2.31,197.33,36,2.25,15,0.94
2,Arsenal,6-0-2,5-2-1,11-2-3,16,36,2.25,192.00,33,2.06,15,0.94
3,Aston Villa,8-0-0,3-3-2,11-3-2,16,35,2.19,186.67,35,2.19,20,1.25
4,Man City,5-0-2,5-3-1,10-3-3,16,33,2.06,176.00,38,2.38,18,1.12
5,Tottenham,5-3-0,4-1-3,9-4-3,16,30,1.88,160.00,33,2.06,23,1.44
6,Man United,5-4-0,4-3-0,9-7-0,16,27,1.69,144.00,18,1.12,21,1.31
